In [1]:
import numpy as np
import pandas as pd 

In [2]:
train = pd.read_csv("train.csv")
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0
...,...,...,...,...,...,...,...,...,...
52555,1094,21,30,0,0,2.4,70.70,-4,0.0
52556,1094,22,0,0,0,2.4,66.79,-4,0.0
52557,1094,22,30,0,0,2.2,66.78,-4,0.0
52558,1094,23,0,0,0,2.1,67.72,-4,0.0


In [3]:
#hour angle 추가
h_list=[]
len(train['Hour'])
for i in range(len(train['Hour'])):
  h=train.loc[i,'Hour']
  min=train.loc[i,'Minute']
  if min!=30:
    hour_angle=15*(h-12)-7.5
    h_list.append(hour_angle)
  else:
    hour_angle=15*(h-12)
    h_list.append(hour_angle)

train['h']=h_list

In [ ]:
# Time 변수 생성 
train['Time'] = train['Hour'] * 60 + train['Minute']
train.to_csv('train.csv')
train.info()

In [5]:
# 날짜별 데이터프레임 쪼개기
train_day = []
list_day = [] 
DAY = 0

for n in range(0,train.shape[0]):

    if train.loc[n, 'Day'] == DAY:
        list_day.append(train.loc[n, ['Time','TARGET']])
        
    else:
        day = pd.DataFrame(list_day, columns = ['Time', 'TARGET'], index = range(0,48))
        train_day.append(day)

        list_day = []
        list_day.append(train.loc[n, ['Time','TARGET']])
    DAY = train.loc[n, 'Day']



In [6]:
# 일출시간, 일몰시간 구하기
new = pd.DataFrame(columns = ['Day', '일출시간', '일몰시간', '일조시간'])

for i in range(0,len(train_day)):
    for d1 in range(0,48):
        if train_day[i].loc[d1,'TARGET'] != 0:
            sunrise = train_day[i].loc[d1,'Time']
            break

    for d2 in range(47,-1,-1):
        if train_day[i].loc[d2,'TARGET'] != 0:
            sunset = train_day[i].loc[d2 + 1,'Time']
            break
        
    new.loc[i] = [i, sunrise, sunset, sunset - sunrise]

new.to_csv('new.csv')

In [7]:
new_train = pd.read_csv('new.csv')

In [8]:
new_train.head()

,Unnamed: 0,Day,일출시간,일몰시간,일조시간
0,0,0.0,480.0,1020.0,540.0
1,1,1.0,480.0,1020.0,540.0
2,2,2.0,480.0,1020.0,540.0
3,3,3.0,480.0,1020.0,540.0
4,4,4.0,480.0,1020.0,540.0


In [9]:
# 하지, 동지 구하기

summer = new.loc[new['일조시간'].idxmax(), 'Day']
winter = new.loc[new['일조시간'].idxmin(), 'Day']

print(summer, winter)

# 138 + 195 = 233 -> 1월 1일

138.0 356.0


In [10]:
N=[]
#1월 1일 기준 N+171 = 하지 기준 N
for i in range(len(train['Day'])):
  day=train.loc[i,'Day']
  if day==summer:
    N.append(+171)
  else:
    N.append(day-summer+171)


#N Feature 생성
#N이 음수, 364 초과를 변환(1월 1일 기준 N=0)
train['N']=N
day_max=train['N'].max()
day_min=train['N'].min()

while day_max>364 or day_max<0:
  for i in range(len(train['N'])):
    day=train.loc[i,'N']
    if day<0:
      train.loc[i,'N']=day+365
    elif day>364:
      train.loc[i,'N']=day-365
    else:
      pass
  day_max=train['N'].max()
  day_min=train['N'].min()

#현재 N은 한국 기준이므로, UT로 변환
train['N']-=0.375


In [11]:
#Declination of the Sun

def radian_angle(x):
  return x*np.pi/180

def dec_sun(N):
  return -1*np.arcsin(0.39779*np.cos(radian_angle(0.98565)*(N+10)+radian_angle(1.914)*np.sin(radian_angle(0.98565)*(N-2))))

train['Declination']=dec_sun(train['N'])

In [12]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0,32.625,-0.295314
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30,32.625,-0.295314
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60,32.625,-0.295314
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90,32.625,-0.295314
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120,32.625,-0.295314


In [13]:
#Zenith Angle

#관측자 위도 설정
alt=37.54

def zenith_angle(altitude,declination,hour_angle):
  return np.sin(radian_angle(altitude))*np.sin(declination)+np.cos(radian_angle(altitude))*np.cos(declination)*np.cos(radian_angle(hour_angle))

train['Zenith_Angle']=zenith_angle(alt,train['Declination'],train['h'])

In [14]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0,32.625,-0.295314,-0.929448
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30,32.625,-0.295314,-0.935938
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60,32.625,-0.295314,-0.929448
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90,32.625,-0.295314,-0.910089
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120,32.625,-0.295314,-0.878193


In [15]:
#GHI

train['GHI']=train['DHI']+train['DNI']*train['Zenith_Angle']

In [16]:
train.head(226)

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle,GHI
0,0,0,0,0,0,1.5,69.08,-12,0.000000,-187.5,0,32.625,-0.295314,-0.929448,0.000000
1,0,0,30,0,0,1.5,69.06,-12,0.000000,-180.0,30,32.625,-0.295314,-0.935938,0.000000
2,0,1,0,0,0,1.6,71.78,-12,0.000000,-172.5,60,32.625,-0.295314,-0.929448,0.000000
3,0,1,30,0,0,1.6,71.75,-12,0.000000,-165.0,90,32.625,-0.295314,-0.910089,0.000000
4,0,2,0,0,0,1.6,75.20,-12,0.000000,-157.5,120,32.625,-0.295314,-0.878193,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
221,4,14,30,142,100,7.1,74.91,5,16.422476,30.0,870,36.625,-0.274578,0.495762,191.576179
222,4,15,0,96,4,6.9,74.50,5,9.196587,37.5,900,36.625,-0.274578,0.440297,97.761187
223,4,15,30,68,0,6.4,80.02,4,6.381375,45.0,930,36.625,-0.274578,0.374471,68.000000
224,4,16,0,38,0,6.0,78.19,4,3.566063,52.5,960,36.625,-0.274578,0.299412,38.000000


In [17]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [18]:
data_path = '/content/drive/MyDrive/'
test = pd.read_csv(data_path+'test/0.csv')
for i in range(1,81):
    data_path1 = data_path + 'test/' + str(i) +'.csv'
    test1 = pd.read_csv(data_path1)
    test = test.append(test1, ignore_index = True)

In [ ]:
test.tail()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
27211,6,21,30,0,0,0.8,63.35,13.7,0.0
27212,6,22,0,0,0,0.7,64.82,13.1,0.0
27213,6,22,30,0,0,0.7,66.10,12.8,0.0
27214,6,23,0,0,0,0.6,67.64,12.4,0.0
27215,6,23,30,0,0,0.6,69.44,12.0,0.0


In [19]:
#hour angle 추가
h_list=[]
len(test['Hour'])
for i in range(len(test['Hour'])):
  h=test.loc[i,'Hour']
  min=test.loc[i,'Minute']
  if min!=30:
    hour_angle=15*(h-12)-7.5
    h_list.append(hour_angle)
  else:
    hour_angle=15*(h-12)
    h_list.append(hour_angle)

test['h']=h_list

In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h
0,0,0,0,0,0,2.7,34.42,0.0,0.0,-187.5
1,0,0,30,0,0,2.7,34.17,0.1,0.0,-180.0
2,0,1,0,0,0,2.7,34.23,0.2,0.0,-172.5
3,0,1,30,0,0,2.7,33.99,0.3,0.0,-165.0
4,0,2,0,0,0,2.8,33.97,0.4,0.0,-157.5


In [20]:
test['Time'] = test['Hour'] * 60 + test['Minute']
test.to_csv('test.csv')
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27216 entries, 0 to 27215
Data columns (total 11 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Day     27216 non-null  int64  
 1   Hour    27216 non-null  int64  
 2   Minute  27216 non-null  int64  
 3   DHI     27216 non-null  int64  
 4   DNI     27216 non-null  int64  
 5   WS      27216 non-null  float64
 6   RH      27216 non-null  float64
 7   T       27216 non-null  float64
 8   TARGET  27216 non-null  float64
 9   h       27216 non-null  float64
 10  Time    27216 non-null  int64  
dtypes: float64(5), int64(6)
memory usage: 2.3 MB


In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time
0,0,0,0,0,0,2.7,34.42,0.0,0.0,-187.5,0
1,0,0,30,0,0,2.7,34.17,0.1,0.0,-180.0,30
2,0,1,0,0,0,2.7,34.23,0.2,0.0,-172.5,60
3,0,1,30,0,0,2.7,33.99,0.3,0.0,-165.0,90
4,0,2,0,0,0,2.8,33.97,0.4,0.0,-157.5,120


In [21]:
# 날짜별 데이터프레임 쪼개기
test_day = []
list_day_1 = [] 
DAY = 0

for n in range(0,test.shape[0]):

    if test.loc[n, 'Day'] == DAY:
        list_day_1.append(test.loc[n, ['Time','TARGET']])
        
    else:
        day = pd.DataFrame(list_day_1, columns = ['Time', 'TARGET'], index = range(0,48))
        test_day.append(day)

        list_day_1 = []
        list_day_1.append(test.loc[n, ['Time','TARGET']])
    DAY = test.loc[n, 'Day']



In [22]:
# 일출시간, 일몰시간 구하기
new_1 = pd.DataFrame(columns = ['Day', '일출시간', '일몰시간', '일조시간'])

for i in range(0,len(test_day)):
    for d1 in range(0,48):
        if test_day[i].loc[d1,'TARGET'] != 0:
            sunrise = test_day[i].loc[d1,'Time']
            break

    for d2 in range(47,-1,-1):
        if test_day[i].loc[d2,'TARGET'] != 0:
            sunset = test_day[i].loc[d2 + 1,'Time']
            break
        
    new_1.loc[i] = [i, sunrise, sunset, sunset - sunrise]

new_1.to_csv('new_1.csv')

In [23]:
# 하지, 동지 구하기

test_summer = new_1.loc[new_1['일조시간'].idxmax(), 'Day']
test_winter = new_1.loc[new_1['일조시간'].idxmin(), 'Day']

print(test_summer, test_winter)

# 138 + 195 = 233 -> 1월 1일

183.0 0.0


In [24]:
N=[]
#1월 1일 기준 N+171 = 하지 기준 N
for i in range(len(test['Day'])):
  day=test.loc[i,'Day']
  if day==test_summer:
    N.append(+171)
  else:
    N.append(day-test_summer+171)


#N Feature 생성
#N이 음수, 364 초과를 변환(1월 1일 기준 N=0)
test['N']=N
day_max=test['N'].max()
day_min=test['N'].min()

while day_max>364 or day_max<0:
  for i in range(len(test['N'])):
    day=test.loc[i,'N']
    if day<0:
      test.loc[i,'N']=day+365
    elif day>364:
      test.loc[i,'N']=day-365
    else:
      pass
  day_max=test['N'].max()
  day_min=test['N'].min()

#현재 N은 한국 기준이므로, UT로 변환
test['N']-=0.375


In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N
0,0,0,0,0,0,2.7,34.42,0.0,0.0,-187.5,0,352.625
1,0,0,30,0,0,2.7,34.17,0.1,0.0,-180.0,30,352.625
2,0,1,0,0,0,2.7,34.23,0.2,0.0,-172.5,60,352.625
3,0,1,30,0,0,2.7,33.99,0.3,0.0,-165.0,90,352.625
4,0,2,0,0,0,2.8,33.97,0.4,0.0,-157.5,120,352.625


In [25]:
test['Declination']=dec_sun(test['N'])

In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination
0,0,0,0,0,0,2.7,34.42,0.0,0.0,-187.5,0,352.625,-0.408491
1,0,0,30,0,0,2.7,34.17,0.1,0.0,-180.0,30,352.625,-0.408491
2,0,1,0,0,0,2.7,34.23,0.2,0.0,-172.5,60,352.625,-0.408491
3,0,1,30,0,0,2.7,33.99,0.3,0.0,-165.0,90,352.625,-0.408491
4,0,2,0,0,0,2.8,33.97,0.4,0.0,-157.5,120,352.625,-0.408491


In [26]:
test['Zenith_Angle']=zenith_angle(alt,test['Declination'],test['h'])

In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle
0,0,0,0,0,0,2.7,34.42,0.0,0.0,-187.5,0,352.625,-0.408491,-0.963497
1,0,0,30,0,0,2.7,34.17,0.1,0.0,-180.0,30,352.625,-0.408491,-0.969722
2,0,1,0,0,0,2.7,34.23,0.2,0.0,-172.5,60,352.625,-0.408491,-0.963497
3,0,1,30,0,0,2.7,33.99,0.3,0.0,-165.0,90,352.625,-0.408491,-0.944927
4,0,2,0,0,0,2.8,33.97,0.4,0.0,-157.5,120,352.625,-0.408491,-0.914330


In [27]:
test['GHI']=test['DHI']+test['DNI']*test['Zenith_Angle']

In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle,GHI
0,0,0,0,0,0,2.7,34.42,0.0,0.0,-187.5,0,352.625,-0.408491,-0.963497,0.0
1,0,0,30,0,0,2.7,34.17,0.1,0.0,-180.0,30,352.625,-0.408491,-0.969722,0.0
2,0,1,0,0,0,2.7,34.23,0.2,0.0,-172.5,60,352.625,-0.408491,-0.963497,0.0
3,0,1,30,0,0,2.7,33.99,0.3,0.0,-165.0,90,352.625,-0.408491,-0.944927,0.0
4,0,2,0,0,0,2.8,33.97,0.4,0.0,-157.5,120,352.625,-0.408491,-0.914330,0.0


In [ ]:
test['TARGET'][17]

6.19359914

In [ ]:
test.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle,GHI
0,0,0,0,0,0,2.7,34.42,0.0,0,-187.5,0,352.625,-0.408491,-0.963497,0.0
1,0,0,30,0,0,2.7,34.17,0.1,0,-180.0,30,352.625,-0.408491,-0.969722,0.0
2,0,1,0,0,0,2.7,34.23,0.2,0,-172.5,60,352.625,-0.408491,-0.963497,0.0
3,0,1,30,0,0,2.7,33.99,0.3,0,-165.0,90,352.625,-0.408491,-0.944927,0.0
4,0,2,0,0,0,2.8,33.97,0.4,0,-157.5,120,352.625,-0.408491,-0.914330,0.0


In [29]:
train.head()

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET,h,Time,N,Declination,Zenith_Angle,GHI
0,0,0,0,0,0,1.5,69.08,-12,0.0,-187.5,0,32.625,-0.295314,-0.929448,0.0
1,0,0,30,0,0,1.5,69.06,-12,0.0,-180.0,30,32.625,-0.295314,-0.935938,0.0
2,0,1,0,0,0,1.6,71.78,-12,0.0,-172.5,60,32.625,-0.295314,-0.929448,0.0
3,0,1,30,0,0,1.6,71.75,-12,0.0,-165.0,90,32.625,-0.295314,-0.910089,0.0
4,0,2,0,0,0,1.6,75.20,-12,0.0,-157.5,120,32.625,-0.295314,-0.878193,0.0


In [43]:
x_train=train.drop(['Day', 'Hour', 'Minute', 'WS', 'TARGET',
                    'h','N','Declination','Zenith_Angle'], axis=1)
y_train=train['TARGET']
x_test=test.drop(['Day', 'Hour', 'Minute', 'WS', 'TARGET',
                  'h','N','Declination','Zenith_Angle','TARGET'], axis=1)
y_test=test['TARGET']

In [44]:
import lightgbm as lgb

clf = lgb.LGBMRegressor()
clf.fit(x_train, y_train)

LGBMRegressor()

In [45]:
y_pred = clf.predict(x_test)

In [46]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
y_pred_train = clf.predict(x_train)
y_pred_test = clf.predict(x_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
print('mae_train(mae): ', mae_train)
r2_train = r2_score(y_train, y_pred_train)
print('r2_score(r2): ', r2_train)
print('')
mae_test = mean_absolute_error(y_test, y_pred_test)
print('mae_test(mae): ', mae_test)
r2_test = r2_score(y_test, y_pred_test)
print('r2_score(r2): ', r2_test)

mae_train(mae):  0.45358258236982063
r2_score(r2):  0.9986309372332903

mae_test(mae):  3.7428476069325964
r2_score(r2):  0.9074393834372854


In [47]:
print(y_pred_test[17])

6.650993633442511


In [48]:
print(y_test[17])

6.19359914


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics

x_train=train(['Day', 'Hour', 'Minute', 'WS', 'TARGET','h','N','Declination','Zenith_Angle'], axis=1)
y_train=train['TARGET']
x_test=test.drop(['Day', 'Hour', 'Minute', 'WS', 'TARGET','h','N','Declination','Zenith_Angle','TARGET_1'], axis=1)
y_test=test['TARGET']

In [ ]:
x_test.head()

In [ ]:
rfr = RandomForestRegressor()
rfr.fit(x_train, y_train)

In [ ]:
import numpy as np
from sklearn.metrics import mean_absolute_error, r2_score
y_pred_train = rfr.predict(x_train)
y_pred_test = rfr.predict(x_test)

mae_train = mean_absolute_error(y_train, y_pred_train)
print('mae_train(mae): ', mae_train)
r2_train = r2_score(y_train, y_pred_train)
print('r2_score(r2): ', r2_train)
print('')
mae_test = mean_absolute_error(y_test, y_pred_test)
print('mae_test(mae): ', mae_test)
r2_test = r2_score(y_test, y_pred_test)
print('r2_score(r2): ', r2_test)

In [ ]:
print(y_pred_train[17])